In [2]:
!pip install selenium
!pip install webdriver-manager

  Using cached urllib3-2.2.3-py3-none-any.whl (126 kB)
  Attempting uninstall: typing-extensions
    Found existing installation: typing-extensions 3.7.4.3
    Uninstalling typing-extensions-3.7.4.3:
      Successfully uninstalled typing-extensions-3.7.4.3
  Attempting uninstall: attrs
    Found existing installation: attrs 20.3.0
    Uninstalling attrs-20.3.0:
      Successfully uninstalled attrs-20.3.0
  Attempting uninstall: certifi
    Found existing installation: certifi 2020.6.20
    Uninstalling certifi-2020.6.20:
      Successfully uninstalled certifi-2020.6.20
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.25.11
    Uninstalling urllib3-1.25.11:
      Successfully uninstalled urllib3-1.25.11


ERROR: After October 2020 you may experience errors when installing or updating packages. This is because pip will change the way that it resolves dependency conflicts.

We recommend you use --use-feature=2020-resolver to test your packages with the new resolver before it becomes the default.

requests 2.24.0 requires urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1, but you'll have urllib3 2.2.3 which is incompatible.


  Attempting uninstall: urllib3
    Found existing installation: urllib3 2.2.3
    Uninstalling urllib3-2.2.3:
      Successfully uninstalled urllib3-2.2.3


ERROR: After October 2020 you may experience errors when installing or updating packages. This is because pip will change the way that it resolves dependency conflicts.

We recommend you use --use-feature=2020-resolver to test your packages with the new resolver before it becomes the default.

selenium 4.27.1 requires urllib3[socks]<3,>=1.26, but you'll have urllib3 1.25.11 which is incompatible.


In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import time
import json

chrome_driver_path = r"C:\Users\ASUS\Desktop\Assignment_May25\chromedriver-win64\chromedriver.exe"

options = Options()

options.add_argument("--disable-gpu")
options.add_argument("--no-sandbox")
options.add_argument("--window-size=1920,1080") 

service = Service(executable_path=chrome_driver_path)
driver = webdriver.Chrome(service=service, options=options)


driver.get("https://rera.odisha.gov.in/projects/project-list")
time.sleep(5)


soup = BeautifulSoup(driver.page_source, "html.parser")
project_links = []
for a_tag in soup.select('a[title="View Details"]')[:6]:
    link = a_tag.get("href")
    if link:
        full_link = "https://rera.odisha.gov.in" + link
        project_links.append(full_link)


projects_data = []

for idx, url in enumerate(project_links, 1):
    driver.get(url)
    time.sleep(5)  # Allow page to load
    soup = BeautifulSoup(driver.page_source, "html.parser")

    data = {}

    try:
        data['RERA Regd. No'] = soup.select_one("span#ContentPlaceHolder1_lblRegistrationNumber").text.strip()
    except:
        data['RERA Regd. No'] = 'Not found'

    try:
        data['Project Name'] = soup.select_one("span#ContentPlaceHolder1_lblProjectName").text.strip()
    except:
        data['Project Name'] = 'Not found'

   
    try:
        promoter_tab = driver.find_element("xpath", "//a[text()='Promoter Details']")
        promoter_tab.click()
        time.sleep(2)
        soup = BeautifulSoup(driver.page_source, "html.parser")
    except:
        print(f" Could not click 'Promoter Details' for Project {idx}")

    try:
        data['Promoter Name'] = soup.select_one("span#ContentPlaceHolder1_lblPromoterName").text.strip()
    except:
        data['Promoter Name'] = 'Not found'

    try:
        data['Registered Office Address'] = soup.select_one("span#ContentPlaceHolder1_lblRegAddress").text.strip()
    except:
        data['Registered Office Address'] = 'Not found'

    try:
        data['GST No'] = soup.select_one("span#ContentPlaceHolder1_lblGSTNumber").text.strip()
    except:
        data['GST No'] = 'Not found'

    data['Detail Page'] = url
    projects_data.append(data)


print("\n🎉 Final Project Data:\n")
for i, project in enumerate(projects_data, 1):
    print(f"Project {i}")
    for key, value in project.items():
        print(f"{key}: {value}")
    print("-" * 40)

with open("projects_data.json", "w", encoding="utf-8") as f:
    json.dump(projects_data, f, indent=4)

print("\n✅ Data saved to projects_data.json")
